# Sequential 모델을 넘어서: 케라스의 함수형 API

## 함수형 API 소개

In [ ]:
from keras import Input, layers

input_tensor = Input(shape=(32,))
dense = layers.Dense(32, activation='relu')

output_tensor = dense(input_tensor)

## 다중 입력 모델

In [3]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)
question_input = Input(shape=(None,), dtype='int32', name='question')

embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['acc'])

In [ ]:
import numpy as np
from keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size,
                        size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size,
                            size=(num_samples, max_length))
answers = np.random.randint(0, answer_vocabulary_size, size=num_samples)
answers = to_categorical(answers)

model.fit([text, question], answers, epochs=10, batch_size=128)

model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)

## 다중 출력 모델

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                activation='softmax',
                                name='income')(x)
gender_perdiction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

In [ ]:
model.compile(optimizer='rmsprop',
             loss={'age': 'mse', 'income': 'categorical_crossentropy', 'gender', 'binary_crossentropy'},
             loss_weights={'age': 0.25, 'income': 1., 'gender': 10.})

In [ ]:
model.fit(posts, {'age': age_targets, 'income': income_targets, 'gender': gender_targets}, epochs=10, batch_size=64)

## 층으로 구성된 비순환 유향 그래프

인셉션은 합성곱 신경망에서 인기 있는 네트워크 구조입니다. 일찍이 네트워크 안의 네트워크 구조에서 영감을 받아 2013~2014년에 만들었습니다.

In [ ]:
from keras import layers
branch_a = layers.Conv2D(128, 1, activation='relu', strides=2)(x)

branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', stride)(branch_b)

branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation='relu')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)
output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=1)

잔차 연결은 엑셥션을 포함하여 2015년 이후 등장한 많은 네트워크 구조에 있는 그래프 형태의 네트워크 컴포넌트입니다. 대규모 딥러닝 모델에서 흔히 나타나는 두 가지 문제인 그래디언트 소실과 표현 병목을 해결했습니다. 일반적으로 10개 층 이상을 가진 모델에 잔차 연결을 추가하면 도움이 됩니다.

In [ ]:
from keras import layers

y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

y = layers.add([y, x]) # layers.Add()([y, x]) 와 동일함

In [ ]:
from keras import layers

y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)

y = layers.add([y, residual])

## 층 가중치 공유

함수형 API의 중요한 또 하나의 기능은 층 객체를 여러 번 재사용할 수 있다는 것입니다. 층 객체를 두 번 호출하면 새로운 층 객체를 만들지 않고 각 호출에 동일한 가중치를 재사용합니다. 이런 기능 때문에 공유 가지를 가진 모델을 만들 수 있습니다. 이런 가지는 같은 가중치를 공유하고 같은 연산을 수행합니다. 다시 말해 같은 표현을 공유하고 이런 표현은 다른 입력에서 함께 학습합니다.

In [ ]:
# 공유 LSTM 모델

from keras import layers
from keras import Input
from keras.models import Model

lstm = layers.LSTM(32)
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output, right_output], axis=1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

model = Model([left_input, right_input], predictions)
model.fit([left_data, right_data], targets)

## 층과 모델

In [ ]:
from keras import layers
from keras import applications
from keras import Input

xception_base = applications.Xception(weights=None,
                                     include_top=False)

left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

left_features = xception_base(left_input)
right_features = xception_base(right_input)

merged_features = layers.concatenate([left_features, right_features], axis=1)

## 정리

- 차례대로 층을 쌓는 것 이상이 필요할 때는 Sequential API를 사용하지 않습니다.
- 함수형 API를 사용하여 다중 입력, 다중 출력, 복잡한 네트워크 토폴로지를 갖는 케라스 모델을 만드는 방법
- 다른 네트워크 가지에서 같은 층이나 모델 객체를 여러 번 호출하여 가중치를 재사용하는 방법

# 케라스 콜백과 텐서보드를 사용한 딥러닝 모델 검사와 모니터링

## 콜백을 사용하여 모델의 훈련 과정 제어하기

- 모델 체크포인트 저장 : 훈련하는 동안 어떤 지점에서 모델의 현재 가중치를 저장합니다.
- 조기 종료 : 검증 손실이 더 이상 향상되지 않을 때 훈련을 중지합니다(물론 훈련하는 동안 얻은 가장 좋은 모델을 저장합니다).
- 훈련하는 동안 하이퍼파라미터 값을 동적으로 조정합니다 : 옵티마이저의 학습률 같은 경우입니다.
- 훈련과 검증 지표를 로그에 기록하거나 모델이 학습한 표현이 업데이트될 때마다 시각화합니다 : 앞서 보았던 케라스의 진행 표시줄이 하나의 콜백입니다!

In [ ]:
import keras

callbacks_list = [
    keras.callbacks.EarlyStopping(
    monitor='val_acc',
    patience=1),
    keras.callbacks.ModelCheckpoint(
    filepath='my_model.h5',
    monitor='val_loss',
    save_best_only=True)
]

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])

model.fit(x, y,
         epochs=10,
         batch_size=32,
         callbacks=callbacks_list,
         validation_data=(x_val, y_val))

In [ ]:
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1, # 콜백이 호출될 때 학습률을 10배로 줄인다
    patience=10)
]

model.fit(x, y,
         epochs=10,
         batch_size=32,
         callbacks=callbacks_list,
         validation_data=(x_val, y_val))

In [ ]:
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    
    def set_model(self, model):
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        self.activation_model = keras.models.Model(model.input,
                                                  layer_outputs)
        
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
            
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model_predict(validation_sample)
        f = open('activations_at_epoch_' + str(epoch) + '.npz', 'wb')
        np.savez(f, activations)
        f.close()

## 텐서보드 소개 : 텐서플로의 시각화 프레임워크

In [3]:
# %load np_load_allow_pickle_True.py
import numpy as np 
np_load_old = np.load
np.load = lambda *a, **k: np_load_old(*a, allow_pickle=True, **k)

# np.load 사용 후 반드시 아래 코드를 실행할 것!!!
# np.load = np_load_old

In [4]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,
                          input_length=max_len,
                          name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc'])





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________


In [5]:
mkdir my_log_dir

In [ ]:
callbacks = [
    keras.callbacks.TensorBoard(
    log_dir='my_log_dir',
    histogram_freq=1,
    embeddings_freq=1)
]

history = model.fit(x_train, y_train,
                   epochs=20,
                   batch_size=128,
                   validation_split=0.2,
                   callbacks=callbacks)

In [ ]:
tensorboard --logdir=my_log_dir

In [ ]:
from keras.utils import plot_model

plot_model(model, to_file='model.png')

## 정리
- 케라스 콜백은 훈련하는 동안 모델을 모니터링하고 모델 상태를 바탕으로 자동으로 작업을 수행하는 손쉬운 방법입니다.
- 텐서플로를 사용하면 텐서보드를 이용하여 모델 상황을 브라우저에서 시각화할 수 있습니다. 케라스 모델에서는 TensorBoard 콜백을 통해 사용합니다.

# 모델의 성능을 최대로 끌어올리기

## 고급 구조 패턴

정규화는 머신 러닝 모델에 주입되는 샘플들을 균일하게 만드는 광범위한 방법입니다. 배치 정규화의 주요 효과는 잔차 연결과 매우 흡사하게 그래디언트 전파를 도와주는 것입니다. 결국 더 깊은 네트워크를 구성할 수 있습니다. 매우 깊은 네트워크라면 여러 개의 BatchNormalization 층을 포함해야 훈련할 수 있습니다.

In [ ]:
conv_model.add(layers.Conv2D(32, 3, activation='relu'))
conv_model.add(layers.BatchNormalization())

dense_model.add(layers.Dense(32, activation='relu'))
dense_model.add(layers.BatchNormalization())

깊이별 분리 합성곱 : 깊이별 합성곱 다음에 점별 합성곱이 뒤따른다

In [ ]:
from keras.models import Sequential, Model
from keras import layers

height = 64
width = 64
channels = 3
num_classes = 10

model = Sequential()
model.add(layers.SeparableConv2D(32, 3,
                                activation='relu',
                                input_shape=(height, width, channels,)))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

## 하이퍼파라미터 최적화

1. 일련의 하이퍼파라미터를 선택합니다.
2. 선택된 하이퍼파라미터로 모델을 만듭니다.
3. 훈련 데이터에 학습하고 검증 데이터에서 최종 성능을 측정합니다.
4. 다음으로 시도할 하이퍼파라미터를 선택합니다.
5. 이 과정을 반복합니다.
6. 마지막으로 테스트 데이터에서 성능을 측정합니다.

가장 단순하지만 종종 랜덤 탐색이 제일 좋은 방법일 때가 많습니다. Hyperopt라는 라이브러리가 있다. Hyperas도 있다

## 모델 앙상블

분류기를 앙상블하는 좋은 방법은 검증 데이터에서 학습된 가중치를 사용하여 가중 평균하는 것입니다. 좋은 앙상블 가중치를 찾기 위해 랜덤 서치나 넬더-미드 방법 같은 간단한 최적화 알고리즘을 사용할 수 있습니다. 다양성이 앙상블의 힘입니다. 실전에서 잘 동작하는 한 가지 방법은 트리 기반 모델이나 심층 신경망을 앙상블 하는 것입니다. 최근에 실전에서 매우 성공적으로 사용되는 기본 앙상블 스타일은 딥러닝과 얕은 모델을 섞은 넓고 깊은 모델입니다.

## 정리

- 고성능 심층 컨브넷을 만들려면 잔차 연결, 배치 정규화, 깊이별 분리 합성곱을 사용해야 합니다. 미래에는 깊이별 분리 합성곱이 일반적인 합성곱을 완전히 대체할 것입니다. 애플리케이션이 1D나 2D 또는 3D인지와 상관없이 아주 효율적으로 표현을 학습하기 때문입니다.
- 심층 네트워크를 만들 때 많은 하이퍼파라미터와 네트워크 구조를 선택해야 합니다. 이 값들이 모여 모델의 성능을 결정합니다. 이런 선택을 직관이나 랜덤한 선택에 의존하지 않고 최적의 선택을 찾기 위해 하이퍼파라미터 공간을 조직적으로 탐색하는 것이 좋습니다. 현재는 이 과정에 비용이 많이 들고 좋은 도구도 없습니다. 하지만 Hyperopt와 Hyperas 라이브러리가 도움이 될 수 있습니다. 하이퍼파라미터 최적화를 할 때 검증 세트에 과대적합된다는 것을 잊지 마세요!
- 머신 러닝 경연 대회에서 우승하거나 어떤 문제에서 최상위 결과를 얻으려면 대규모로 모델을 앙상블하게 됩니다. 최적화가 잘된 가중 평균으로 만든 앙상블은 보통 충분히 좋은 결과를 만듭니다. 다양성이 중요하다는 것을 기억하세요. 비슷한 모델을 앙상블하는 것은 거의 쓸모가 없습니다. 가장 좋은 앙상블은 가능한 서로 다른 모델로 만드는 것입니다.

# 요약
- 임의의 층 그래프를 구성하는 모델을 만드는 방법, 층을 재사용하는 방법(가중치 공유), 파이썬 함수 방식으로 모델을 사용하는 방법(모델 템플릿)
- 케라스 콜백을 사용하여 훈련하는 동안 모델을 모니터링하고 모델 상태를 바탕으로 작업을 수행합니다.
- 텐서보드를 사용하여 측정 지표, 활성화 출력의 히스토그램, 임베딩 공간을 시각화합니다.
- 배치 정규화, 깊이별 분리 합성곱, 잔차 연결
- 하이퍼파라미터 최적화와 모델 앙상블을 사용하는 이유